In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Nhập dữ liệu 

In [2]:
# data_train = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
# data_test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

data_train = pd.read_csv("train.csv")
data_test = pd.read_csv("test.csv")

In [ ]:
print("dữ liệu huấn luyện: \n");
data_train.info


In [ ]:
data_train.head(2)

In [ ]:

print("\n dữ liệu test: \n");
data_test.info

In [ ]:
data_test.head(2)

Tiền xử lý dữ liệu

In [ ]:
def pourcentage(data):
    n = data.shape[0]
    ret = pd.DataFrame(data.isnull().sum(), columns=['missing_number'])
    ret['types'] = data.dtypes # xác đinh kiểu dữ liệu
    ret['duplicate'] = data.duplicated(keep=False).sum() # xác định các hàng trùng lặp
    ret['NAN'] = data.isna().sum() # tổng số các dòng NaN
    return ret

In [ ]:
pourcentage(data_train).T

In [ ]:
pourcentage(data_test)

In [ ]:
def compare(train, test):
    dataset_train_nans = train.isnull().sum().to_frame().T # tổng số lượng có đối tượng null trong mỗi hàng 
    dataset_train_nans['DataFrame'] = 'data_train'
    
    dataset_test_nans = test.isnull().sum().to_frame().T
    dataset_test_nans['DataFrame'] = 'data_test'

    combined_data = pd.concat([dataset_train_nans, dataset_test_nans], axis=0, ignore_index=True)
    return combined_data

In [ ]:
compare(data_train,data_test)

In [ ]:
def xac_dinh_cac_cot_kieu_ky_tu(x):
    categorical_columns_train = x.select_dtypes(include=["category","object"]).columns.tolist();
    return categorical_columns_train
def xac_dinh_cac_cot_kieu_so(x):
    numberic_columns_train = x.select_dtypes(include=["float64","int64"]).columns.tolist();
    return numberic_columns_train
   

In [ ]:
print(xac_dinh_cac_cot_kieu_ky_tu(data_train))
cac_cot_kieu_ky_tu_train = xac_dinh_cac_cot_kieu_ky_tu(data_train)
cac_cot_kieu_so_train = xac_dinh_cac_cot_kieu_so(data_train)

In [ ]:
phan_tram_thieu = data_train.isnull().sum() * 100 / len(data_train)
phan_tram_thieu = phan_tram_thieu[phan_tram_thieu > 0].sort_values(ascending=False)
print(phan_tram_thieu)

In [ ]:
loai_cot = phan_tram_thieu[phan_tram_thieu > 0].index.tolist()
phan_tram_thieu.drop(columns=loai_cot, inplace=True) #This line excludes features_to_exclude list of columns from training data
loai_cot.append('Id')
loai_cot.append('SalePrice')

test

In [ ]:
phan_tram_thieu_test = data_test.isnull().sum() * 100 / len(data_test)
phan_tram_thieu_test = phan_tram_thieu_test[phan_tram_thieu_test > 0].sort_values(ascending=False)
print(phan_tram_thieu_test)

In [ ]:
loai_cot_ = phan_tram_thieu_test[phan_tram_thieu_test > 0].index.tolist()
phan_tram_thieu_test.drop(columns=loai_cot_, inplace=True) #This line excludes features_to_exclude list of columns from training data
loai_cot_.append('Id')
loai_cot_.append('SalePrice')

In [ ]:
loai_cot=loai_cot+loai_cot_

In [ ]:
loai_cot =list(set(loai_cot))

In [ ]:
print(loai_cot)

In [ ]:
print()

In [ ]:
cac_cot_kieu_ky_tu_train = [col for col in cac_cot_kieu_ky_tu_train if col not in loai_cot]
cac_cot_kieu_so_train = [col for col in cac_cot_kieu_so_train if col not in loai_cot]

In [ ]:
cac_cot_con_lai = cac_cot_kieu_ky_tu_train+cac_cot_kieu_so_train
cac_cot_con_lai.append("SalePrice")

In [ ]:
data_train_new = data_train[cac_cot_con_lai]

In [ ]:
data_train_new.info

In [ ]:
pourcentage(data_train_new)

In [ ]:
def ma_hoa(x):
    from sklearn.calibration import LabelEncoder
    label_encoder = LabelEncoder()
    # Áp dụng mã hóa nhãn cho cột 'Street_Type'
    data_train_new.loc[:,x] = label_encoder.fit_transform(data_train_new[x])


In [ ]:
for cot in cac_cot_kieu_ky_tu_train:
    ma_hoa(cot)

In [ ]:
data_train_new.columns

test

test

In [ ]:
pourcentage(data_test)

In [ ]:
cac_cot_test = data_test.columns

In [ ]:
cac_cot_con_lai_test = [col for col in cac_cot_test if col not in loai_cot]

In [ ]:
data_test_new = data_test[cac_cot_con_lai_test]

In [ ]:
cac_cot_kieu_ky_tu_test = xac_dinh_cac_cot_kieu_ky_tu(data_test_new)
cac_cot_kieu_so_test = xac_dinh_cac_cot_kieu_so(data_test_new)

In [ ]:
def ma_hoa(x):
    from sklearn.calibration import LabelEncoder
    label_encoder = LabelEncoder()
    # Áp dụng mã hóa nhãn cho cột 'Street_Type'
    data_test_new.loc[:,x] = label_encoder.fit_transform(data_test_new[x])

In [ ]:
for cot in cac_cot_kieu_ky_tu_test:
    ma_hoa(cot)

In [ ]:
data_test_new.head

In [ ]:
data_train_new.head

In [ ]:
dac_diem=['Street', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'RoofStyle', 'RoofMatl', 'ExterQual', 'ExterCond', 'Foundation',
       'Heating', 'HeatingQC', 'CentralAir', 'PavedDrive', 'SaleCondition',
       'MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
       'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']
du_doan=['SalePrice']

In [ ]:
X_train = data_train_new[dac_diem]
y_train = data_train_new[du_doan]

In [ ]:
from sklearn import tree
model=tree.DecisionTreeClassifier(criterion = "entropy",
random_state = 100,
max_depth = 4,
min_samples_leaf = 10)

In [ ]:
model = model.fit(X_train,y_train)

In [ ]:
tree.plot_tree(model)

In [ ]:
print(tree.export_text(model,show_weights = True))

In [ ]:
import graphviz
dot_data = tree.export_graphviz(model, out_file=None)
graph = graphviz.Source(dot_data)
graph.render("data_new_train")
dot_data = tree.export_graphviz(model,out_file = None,
feature_names=dac_diem,
filled =True,
rounded = True )
graph = graphviz.Source(dot_data)
graph

In [ ]:
X_test = data_test_new[dac_diem]

In [ ]:
y_pred = model.predict(X_test)
print("Giá trị mô hình dự đoán được:\n " ,y_pred)

In [ ]:
sample_submission_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')
sample_submission_df['SalePrice'] = model.predict(X_test)
sample_submission_df.to_csv('/kaggle/working/submission.csv', index=False)
sample_submission_df.head()